# Random Forest Regression for Public data

This notebook creates a random forest regression model for private data.
This model should serve as benchmark for for public RFR + kNN classificaition.Hyper-parameter optimization at the end of the notebook.

In [1]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Import functions that read from  INTERIM format and normalize the data
sys.path.append(os.path.abspath('../../src/data'))
from extract_for_model import extract_time_series
from extract_for_model import scale_time_series_single

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.tree import export_graphviz  

from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae

In [2]:
# Import required paths to input files
# Change the file to import if needed
from data_links import soft_pub_IS as input_IS
from data_links import soft_pub_BS as input_BS

In [3]:
# *******************************************************************
# Load selected feautures from INCOME STATEMENTS and BALANCE SHEETS
# Combine the data from these two sourcesinto one dataframe
# *******************************************************************


# Read from INCOME STATMENTS
source = input_IS

feat_IS = ['IQ_EBIT','IQ_TOTAL_OTHER_OPER','IQ_EARNING_CO','IQ_TOTAL_REV','IQ_GP','IQ_OTHER_OPER','IQ_INC_TAX','IQ_NET_INTEREST_EXP']

IS_df = extract_time_series(input_path = source,\
                              features = feat_IS,
                                 steps = 1,\
                              year_min = 2012,\
                              no_shift = False,\
                               no_test = True,\
                             quarterly = True)

# Read from BALANCE SHEETS
feat_BS =  ['IQ_AR','IQ_RE','IQ_TOTAL_ASSETS','IQ_TOTAL_CL','IQ_TOTAL_EQUITY','IQ_TOTAL_LIAB_EQUITY']
source = input_BS
BS_df = extract_time_series(input_path = source,\
                              features = feat_BS,
                                 steps = 1,\
                              year_min = 2012,\
                              no_shift = False,\
                               no_test = True,\
                             quarterly = True)

# Merge dataframe
data_set = IS_df.set_index(['company','year','quarter']).join(BS_df.set_index(['company','year','quarter'])).dropna()

====  extract_time_series metric ====
Size of RAW data: (16403, 11)
Size of RESHAPED data without NA: (8661, 27)
No of companies in RESHAPED data: 356
No of companies with more than 1 datapoint 350
====  extract_time_series metric ====
Size of RAW data: (16029, 9)
Size of RESHAPED data without NA: (12771, 21)
No of companies in RESHAPED data: 481
No of companies with more than 1 datapoint 474


In [4]:
data_set

IQ_EBIT-1  IQ_TOTAL_OTHER_OPER-1  IQ_EARNING_CO-1  \
company     year   quarter                                                      
IQ1004737   2012.0 Q1        4.357645               3.713942         2.118456   
                   Q2       -0.239797               2.170620        -1.337370   
                   Q3        2.164441               1.675047         0.810920   
                   Q4        1.556037               1.753607         0.286793   
            2013.0 Q1        3.101893               4.942206         2.162115   
                   Q2        1.371334               1.712648         0.056548   
                   Q3        1.250708               1.542227         0.401993   
                   Q4        2.067693               1.705344         0.643435   
            2014.0 Q1        2.937156               4.861440         1.287832   
                   Q2        0.819894               1.201428        -0.338367   
                   Q3        1.465513               1.497839         0.514133   
                   Q4        2.408262               1.289644         0.972588   
            2015.0 Q1        3.888034               3.575317         1.206973   
                   Q2        0.886342               0.980421        -0.196194   
                   Q3        2.682166              -1.623996         0.958093   
            2016.0 Q1        3.053093               3.604232         1.987580   
                   Q2        0.635010               1.328073        -0.216279   
IQ100842126 2013.0 Q4        1.074361               0.553854         0.935867   
            2014.0 Q1        1.380804               0.238956         0.931140   
                   Q2       -0.906065               0.689986        -0.957458   
                   Q3       -0.877278               0.668063        -0.927037   
                   Q4        0.289069               0.524245        -0.095911   
            2015.0 Q1        0.288576               0.523352        -0.095747   
                   Q2       -0.562944               0.470747        -0.631278   
                   Q3       -0.551432               0.461120        -0.618369   
                   Q4        1.438522               0.257761         0.796147   
            2016.0 Q1        1.942426              -0.248557         0.795014   
                   Q2        0.068003               0.242754        -0.171193   
                   Q3        0.071840               0.256454        -0.180854   
                   Q4        2.669081              -0.275799         2.220206   
...                               ...                    ...              ...   
IQ98244713  2016.0 Q4        0.141965               0.225070         0.064055   
            2017.0 Q1        0.135915               0.238399         0.116217   
                   Q2        0.205713               0.239679         0.100483   
                   Q3        0.082395               0.291245         0.047176   
                   Q4        0.369046               0.271421         0.291324   
            2018.0 Q1        0.164224               0.282206         0.092501   
                   Q2        0.052266               0.282216         0.050185   
                   Q3        0.309933               0.300732         0.284009   
IQ9898268   2012.0 Q1        0.517235               3.013385         0.344948   
                   Q2        0.339544               2.386238         0.245226   
                   Q3        0.165988               2.369937        -0.516407   
                   Q4        0.122604               2.103134         0.075449   
            2013.0 Q1        0.328446               2.596857         0.310871   
                   Q2        0.312669               2.471092         0.262238   
                   Q4        0.136738               2.392908         0.117204   
            2014.0 Q2        0.582631               2.631840         0.505636   
                   Q3        0.253387               2.29796

In [5]:
# ****************************
# Calculate percentage changes
# ****************************

for feature in feat_IS+feat_BS:
    data_set[feature+'_PERC+1'] = (data_set[feature+'+1']-data_set[feature+'+0'])/data_set[feature+'+0']
    data_set[feature+'_PERC+0'] = (data_set[feature+'+0']-data_set[feature+'-1'])/data_set[feature+'-1']

data_set=data_set.replace(np.nan, 0)
data_set=data_set.replace(np.inf, 0)
data_set.describe()

,IQ_EBIT-1,IQ_TOTAL_OTHER_OPER-1,IQ_EARNING_CO-1,IQ_TOTAL_REV-1,IQ_GP-1,IQ_OTHER_OPER-1,IQ_INC_TAX-1,IQ_NET_INTEREST_EXP-1,IQ_EBIT+0,IQ_TOTAL_OTHER_OPER+0,...,IQ_RE_PERC+1,IQ_RE_PERC+0,IQ_TOTAL_ASSETS_PERC+1,IQ_TOTAL_ASSETS_PERC+0,IQ_TOTAL_CL_PERC+1,IQ_TOTAL_CL_PERC+0,IQ_TOTAL_EQUITY_PERC+1,IQ_TOTAL_EQUITY_PERC+0,IQ_TOTAL_LIAB_EQUITY_PERC+1,IQ_TOTAL_LIAB_EQUITY_PERC+0
count,5445.000000,5445.000000,5445.000000,5445.000000,5445.000000,5445.000000,5445.000000,5445.000000,5445.000000,5445.000000,...,5445.000000,5445.000000,5445.000000,5445.000000,5445.000000,5445.000000,5445.000000,5445.000000,5445.000000,5445.000000
mean,12.875025,22.141303,8.863872,100.822991,35.016327,1.470627,2.875589,-0.358172,12.952368,23.496386,...,0.154823,0.049375,0.046272,0.057629,0.071491,0.089956,-76.715277,-76.737515,0.046272,0.057629
std,87.472509,145.274636,66.443114,487.928221,229.708048,12.591678,20.841533,4.050129,87.902922,149.364989,...,11.660466,13.208824,0.288446,0.413304,0.422456,0.577258,5742.385962,5742.385286,0.288446,0.413304
min,-28.200117,-301.157650,-91.820570,-11.126720,-265.817720,-240.463350,-121.689920,-191.891940,-382.351930,-301.157650,...,-355.845148,-478.176856,-0.776025,-0.972774,-0.982871,-0.993744,-423692.000000,-423692.000000,-0.776025,-0.972774
25%,0.006700,0.490736,-0.040858,2.300858,0.567539,-0.024000,0.001372,-0.137666,0.004267,0.504625,...,-0.047746,-0.061297,-0.027087,-0.028777,-0.052736,-0.055340,-0.021657,-0.023357,-0.027087,-0.028777
50%,0.428496,1.738199,0.259021,8.195864,2.247066,0.049198,0.083596,-0.025630,0.436500,1.779039,...,0.008690,0.007784,0.007784,0.007784,0.007782,0.007392,0.008690,0.007784,0.007784,0.007784
75%,1.965589,6.665706,1.297425,27.614288,8.540306,1.062572,0.498500,0.000094,2.013033,6.811201,...,0.118768,0.118525,0.064469,0.069313,0.112219,0.122895,0.059435,0.061260,0.064469,0.069313
max,1468.934100,2856.169200,1649.737500,6820.077600,4210.073000,407.277830,379.905060,63.469944,1396.875900,2856.169200,...,356.238227,356.238227,8.786149,12.771866,12.876667,19.349667,5741.411535,5741.411535,8.786149,12.771861


In [6]:
# ****************************
# Create filters for data segmentation
# ****************************
res = data_set
target_name = 'IQ_TOTAL_REV+0'

mask1 = res[target_name] < 5
mask2 = (res[target_name] >= 5 ) & (res[target_name] < 10 )
mask3 = (res[target_name] >= 10 ) & (res[target_name] < 50 )
mask4 = (res[target_name] >= 50)

In [7]:
predictors = ['IQ_TOTAL_OTHER_OPER_PERC+0','IQ_TOTAL_REV_PERC+0','IQ_GP_PERC+0',
             'IQ_AR_PERC+0','IQ_RE_PERC+0','IQ_TOTAL_ASSETS_PERC+0', 'IQ_EBIT_PERC+0','IQ_EBIT+0']
target     =  'IQ_EBIT_PERC+1'

In [8]:
masks      = [mask1,mask2,mask3,mask4]
rf = RandomForestRegressor(n_estimators=100,random_state = 0)
columns = ['Segment','MAE','MSE']
segment_summary = pd.DataFrame(columns=columns)

for idx,mask in enumerate(masks):
    segment     = idx+1
    #
    #
    res = data_set[mask].copy()
    X_train, X_test, y_train, y_test = train_test_split(np.array(res[predictors]), np.array(res[target]), test_size=0.33, random_state=42)
    X_previous=X_test[:,7]
    X_train=np.delete(X_train, -1, axis=1)
    X_test=np.delete(X_test, -1,axis=1)
    X_previous
    X_previous=pd.DataFrame(X_previous)  
    #
    rf.fit(X_train, y_train)
    predictions = rf.predict(X_test)
    #
    real_val=(pd.DataFrame(y_test+1))*X_previous
    predicted_real=(pd.DataFrame(predictions)+1)*(X_previous)
    #
    export_graphviz(rf.estimators_[0], out_file='tree_from_forest_private_seg_'+str(idx)+'.dot')
    #
    MSE = mse(real_val,predicted_real)
    MAE = mae(real_val,predicted_real)
    row = pd.Series([segment,MAE,MSE],index=columns)
    segment_summary = segment_summary.append(row,ignore_index=True)

In [9]:
segment_summary

,Segment,MAE,MSE
0,1.0,1.264352,140.380010
1,2.0,1.718169,37.561290
2,3.0,2.000855,23.636986
3,4.0,31.394143,19677.646418


In [10]:
# ****************************
# Hyperparameter optimization
# ****************************
#
#  Varies selected hyper-parameters and collects them to the dictionary
#
n_estimator_range = [100,200,400,800,1600]
criteria = ['mse','mae']
max_depths = [1,3,5,7,9,11]

opti_results ={}
for n_est in n_estimator_range:
    for max_depth in max_depths:
        for criterion in criteria:
            for idx,mask in enumerate(masks):
                segment = idx+1
                #
                #
                res = data_set[mask].copy()
                X_train, X_test, y_train, y_test = train_test_split(np.array(res[predictors]), np.array(res[target]), test_size=0.33, random_state=42)
                X_previous = X_test[:,7]
                X_train = np.delete(X_train, -1, axis=1)
                X_test = np.delete(X_test, -1,axis=1)
                X_previous = pd.DataFrame(X_previous)  
                #
                rf = RandomForestRegressor(n_estimators=n_est,
                                           criterion=criterion,
                                           max_depth=max_depth,
                                           random_state = 0)
                rf.fit(X_train, y_train)
                predictions = rf.predict(X_test)
                #
                real_val=(pd.DataFrame(y_test+1))*X_previous
                predicted_real=(pd.DataFrame(predictions)+1)*(X_previous)
                #
                #export_graphviz(rf.estimators_[0], out_file='tree_from_forest_private_seg_'+str(idx)+'.dot')
                #
                MSE = mse(real_val,predicted_real)
                MAE = mae(real_val,predicted_real)
                row = pd.Series([segment,MAE,MSE])
                #
                key_tuple = (segment,n_est,max_depth,criterion)
                #              
                opti_results[key_tuple] = row
                print("{} : {}".format(key_tuple,row) )             



(1, 100, 1, 'mse') : 0     1.000000
1     0.662039
2    16.206858
dtype: float64
(2, 100, 1, 'mse') : 0     2.000000
1     1.443509
2    20.975910
dtype: float64
(3, 100, 1, 'mse') : 0     3.000000
1     1.915273
2    57.233205
dtype: float64
(4, 100, 1, 'mse') : 0        4.000000
1       34.628613
2    32570.655755
dtype: float64
(1, 100, 1, 'mae') : 0    1.000000
1    0.209611
2    0.183516
dtype: float64
(2, 100, 1, 'mae') : 0    2.000000
1    0.416293
2    0.643557
dtype: float64
(3, 100, 1, 'mae') : 0     3.000000
1     1.282326
2    10.738549
dtype: float64
(4, 100, 1, 'mae') : 0       4.000000
1      25.139885
2    7225.012148
dtype: float64
(1, 100, 3, 'mse') : 0      1.000000
1      1.018176
2    137.135940
dtype: float64
(2, 100, 3, 'mse') : 0     2.000000
1     1.407035
2    33.943336
dtype: float64
(3, 100, 3, 'mse') : 0     3.000000
1     1.733186
2    24.546397
dtype: float64
(4, 100, 3, 'mse') : 0        4.000000
1       31.147421
2    21889.861488
dtype: float64
(1, 100

(1, 400, 1, 'mae') : 0    1.000000
1    0.211520
2    0.185899
dtype: float64
(2, 400, 1, 'mae') : 0    2.000000
1    0.415873
2    0.644588
dtype: float64
(3, 400, 1, 'mae') : 0     3.000000
1     1.348502
2    17.046959
dtype: float64
(4, 400, 1, 'mae') : 0       4.000000
1      24.877624
2    7285.452980
dtype: float64
(1, 400, 3, 'mse') : 0      1.000000
1      1.019834
2    160.816345
dtype: float64
(2, 400, 3, 'mse') : 0     2.000000
1     1.449563
2    35.975807
dtype: float64
(3, 400, 3, 'mse') : 0     3.000000
1     1.793556
2    31.784376
dtype: float64
(4, 400, 3, 'mse') : 0        4.000000
1       30.218099
2    14349.697315
dtype: float64
(1, 400, 3, 'mae') : 0    1.00000
1    0.22307
2    0.20197
dtype: float64
(2, 400, 3, 'mae') : 0    2.000000
1    0.441625
2    0.676101
dtype: float64
(3, 400, 3, 'mae') : 0     3.000000
1     1.370378
2    14.890552
dtype: float64
(4, 400, 3, 'mae') : 0       4.000000
1      23.280818
2    5864.227758
dtype: float64
(1, 400, 5, 'mse') 

(1, 1600, 3, 'mse') : 0      1.000000
1      0.969467
2    131.885391
dtype: float64
(2, 1600, 3, 'mse') : 0     2.000000
1     1.422587
2    38.140846
dtype: float64
(3, 1600, 3, 'mse') : 0     3.000000
1     1.757752
2    29.497837
dtype: float64
(4, 1600, 3, 'mse') : 0        4.000000
1       30.732404
2    15165.610949
dtype: float64
(1, 1600, 3, 'mae') : 0    1.000000
1    0.221565
2    0.198262
dtype: float64
(2, 1600, 3, 'mae') : 0    2.000000
1    0.444858
2    0.672361
dtype: float64
(3, 1600, 3, 'mae') : 0     3.000000
1     1.375126
2    14.840598
dtype: float64
(4, 1600, 3, 'mae') : 0       4.000000
1      22.771139
2    5947.094217
dtype: float64
(1, 1600, 5, 'mse') : 0      1.000000
1      1.029342
2    149.725733
dtype: float64
(2, 1600, 5, 'mse') : 0     2.000000
1     1.471262
2    39.320356
dtype: float64
(3, 1600, 5, 'mse') : 0     3.000000
1     1.781781
2    30.084044
dtype: float64
(4, 1600, 5, 'mse') : 0        4.000000
1       29.267296
2    14438.424470
dtype: 

In [11]:
# Print the results sorted by error for each segment:
columns     = ['Segment','Hyper','MAE','MSE']
opti_sum_pd = pd.DataFrame(columns=columns)

for k,v in sorted(opti_results.items(),key = lambda kv: (kv[1][1],kv[1][2])):
    row = pd.Series([k[0],k[1:],v[1],v[2]],index = columns)
    opti_sum_pd = opti_sum_pd.append(row,ignore_index=True)
    
for segment in opti_sum_pd['Segment'].unique():
    print(opti_sum_pd[opti_sum_pd['Segment']==segment])
    

    Segment            Hyper       MAE         MSE
0         1    (100, 1, mae)  0.209611    0.183516
1         1   (1600, 1, mae)  0.210924    0.185196
2         1    (800, 1, mae)  0.211017    0.185155
3         1    (200, 1, mae)  0.211036    0.185077
4         1    (400, 1, mae)  0.211520    0.185899
5         1   (1600, 3, mae)  0.221565    0.198262
6         1    (400, 3, mae)  0.223070    0.201970
7         1    (800, 3, mae)  0.223705    0.201230
8         1    (200, 3, mae)  0.228461    0.231613
9         1    (800, 5, mae)  0.241523    0.263821
10        1   (1600, 5, mae)  0.241557    0.273073
11        1    (100, 3, mae)  0.241584    0.358502
12        1    (400, 5, mae)  0.244615    0.284873
13        1    (100, 5, mae)  0.245715    0.378680
14        1    (100, 7, mae)  0.254597    0.365081
15        1    (200, 5, mae)  0.258946    0.459791
16        1   (1600, 7, mae)  0.268517    0.580562
17        1    (800, 7, mae)  0.268771    0.573987
18        1    (400, 7, mae)  0